Imports

In [1]:
import pandas as pd

Loading The Dataset

In [6]:
file_path = 'CrossCheck_Daily_Data.xlsx'  
data = pd.read_excel(file_path)

print("Dataset Shape:", data.shape)
print("Column Names:", data.columns)
print(data.head())

Dataset Shape: (23573, 155)
Column Names: Index(['study_id', 'eureka_id', 'day', 'act_in_vehicle_ep_0',
       'act_in_vehicle_ep_1', 'act_in_vehicle_ep_2', 'act_in_vehicle_ep_3',
       'act_in_vehicle_ep_4', 'act_on_bike_ep_0', 'act_on_bike_ep_1',
       ...
       'unlock_duration_ep_0', 'unlock_duration_ep_1', 'unlock_duration_ep_2',
       'unlock_duration_ep_3', 'unlock_duration_ep_4', 'unlock_num_ep_0',
       'unlock_num_ep_1', 'unlock_num_ep_2', 'unlock_num_ep_3',
       'unlock_num_ep_4'],
      dtype='object', length=155)
   study_id eureka_id       day  act_in_vehicle_ep_0  act_in_vehicle_ep_1  \
0        -1      u004  20150122                    0                    0   
1        -1      u004  20150123                    0                    0   
2        -1      u004  20150124                    0                    0   
3        -1      u004  20150125                    0                    0   
4        -1      u004  20150126                    0                    0   

Data Cleaning

In [12]:
#get all coulmns that have all data = zero and drop it 
nullCoulmns=data.columns[(data==0).all()] 
dataCleaned=data.drop(columns=nullCoulmns)

# eureka_id dropped because its redundant and duplicates the study id
dataCleaned=data.drop(columns=['eureka_id'])

print(f"Dropped {len(nullCoulmns)} columns with only zero values.")
print("Remaining columns:", dataCleaned.columns)

Dropped 13 columns with only zero values.
Remaining columns: Index(['study_id', 'day', 'act_in_vehicle_ep_0', 'act_in_vehicle_ep_1',
       'act_in_vehicle_ep_2', 'act_in_vehicle_ep_3', 'act_in_vehicle_ep_4',
       'act_on_bike_ep_0', 'act_on_bike_ep_1', 'act_on_bike_ep_2',
       ...
       'unlock_duration_ep_0', 'unlock_duration_ep_1', 'unlock_duration_ep_2',
       'unlock_duration_ep_3', 'unlock_duration_ep_4', 'unlock_num_ep_0',
       'unlock_num_ep_1', 'unlock_num_ep_2', 'unlock_num_ep_3',
       'unlock_num_ep_4'],
      dtype='object', length=154)
